In [ ]:
import random

class Value:
    def __init__(self, value, _children=()):
        self.value = value
        self.grad = 0
        self._backward = lambda: None
        self._prev = set(_children)
        
    def __repr__(self):
        return "value: {} ; grad: {}".format(self.value, self.grad)
    
    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.value + other.value, (self, other))
        
        def _backward():
            self.grad += out.grad 
            other.grad += out.grad 
        out._backward = _backward
        return out
    
    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.value * other.value, (self, other))
        
        def _backward():
            self.grad += out.grad
            other.grad += out.grad
        out._backward = _backward
        return out
    
    def __sub__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.value-other.value, (self, other))
        
        def _backward():
            self.grad += out.grad
            other.grad += out.grad
        out._backward = _backward
        return out
    
    def __pow__(self, p):
        assert isinstance(p, (float, int))
        out = Value(self.value**p, (self,))
        
        def _backward():
            self.grad += (p * out.grad ** (p-1)) * out.grad
        out._backward = _backward
        return out
    
    def __relu__(self):
        out = Value(0) if self.value < 0 else Value(self.value)
        
        def _backward():
            self.grad += out.value > 0 * out.grad
        out._backward = _backward
        return out
    
    def backward(self):
        topo = []
        visited = set()
        def topological_sort(node):
            if node not in visited:
                visited.add(node)
                for c in node._prev:
                    topological_sort(c)
                topo.append(node)
        topological_sort(self)
        print(topo)
        # Apply the chain rule:
        self.grad = 1
        for n in reversed(topo):
            n._backward()
            
    
    

In [ ]:
a = Value(5)
b = Value(-1)
c = a - b
d = c**2 * c**3
e = d - 2

def viz():
    print(a)
    print(b)
    print(c)
    print(d)

e.backward()
viz()

In [ ]:
# Create a neuron
class Neuron:
    def __init__(self, n):
        self.n = n
        self.w = [Value(random.uniform(-1,1)) for _ in range(self.n)]
        self.b = random.uniform(-1,1)
        self.val = None
    
    def __call__(self, inp):
        print(inp)
        print(self.w)
        act = sum([i.value * w.value for i,w in zip(inp, self.w)])
        out = Value(act).__relu__()
        self.val = out
        return out
        
    def __repr__(self):
        return "self.w: {} | self.b: {} | self.val: {}".format(self.w, self.b, self.val)
        
# Create a layer
# Note: a layer consists of neurons and we are only defining one layer at a time here.
class Layer:
    def __init__(self, neuron_in_dim, no_of_neurons):
        self.no_of_neurons = no_of_neurons
        self.layer = [Neuron(neuron_in_dim) for _ in range(no_of_neurons)]
        
    # def __repr__(self):
    #     return [print(n) for n in self.layer]
    
    def __call__(self, x):
        out = [n(xi) for n, xi in zip(self.layer, x)]
        return out
    
# Create an MLP
# Note: consists of multiple layers, each with a set of neurons
class MLP:
    def __init__(self, input_dim, neurons_per_layer):
        self.input_dim = input_dim
        self.neurons_per_layer = neurons_per_layer
        self.n_layers = len(self.neurons_per_layer)
        self.dims = [input_dim] + neurons_per_layer
        self.layers = [Layer(self.dims[i], self.dims[i+1]) for i in range(len(self.dims)-1)]
        
    def __repr__(self):
        return [print(l) for l in self.layers]
        
    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x



In [ ]:
n = 3
x = [Value(1), Value(1), Value(1)]
Neuron(n)(x)

In [ ]:
mlp = MLP(3, [4, 4, 1])
out = mlp(x)
print(out)